In [1]:
# >>>>>>>>>>>>>>>>> standard libs
import os, os.path
import sys
import time

import shutil

In [2]:
# >>>>>>>>>>>>>>>>> basic 3rd libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# torch libs
import torch
from torch.nn import Linear
import torch.nn as nn
from torch_geometric.nn import GCNConv

# from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader

from torch.utils.data import random_split

In [4]:
# >>>>>>>>> my libs
sys.path.append('../../ai.rdee')
import air

***

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# 1. 首先, 实现air库中的Dataset

In [6]:
class SZD4GRU(air.utils.airDataset, torch.utils.data.Dataset):
    def __init__(self, scaler=None):
        air.utils.airDataset.__init__(self)
        
        for f in self.processed_file_names:
            if not os.path.exists(f):
                self.process()
        self.data = torch.load(self.processed_file_names[0])  # (nSamples, nFeatures, nNodes)
        self._index = list(range(self.data.shape[0]))
        
        self.scalerX = scaler() if scaler is not None else air.utils.NoScaler()
        self.scalerY = self.scalerX

        self.nNodes = self.data.shape[2]
        self.parent = None
        
    @property
    def raw_file_names(self):
        return ['data/sz_adj.csv', 'data/sz_speed.csv']

    @property
    def processed_file_names(self):
        return ['data_sz_GRU.pt']

    def download(self):
        # Download to `self.raw_dir`.
        print("Please download the data manually!")
        sys.exit(0)
        
    def process(self):
        for f in self.raw_file_names:
            if not os.path.exists(f):
                slef.download()
                break
        print("Processing ... ...")
        # Read data into huge `Data` list.
        fn_adj, fn_spd = self.raw_file_names
        df_spd = pd.read_csv(fn_spd)
        S = df_spd.values  # nTimes x nNodes
        
        data = np.zeros((S.shape[0]-14, 15, S.shape[1]))  # (nSamples, nSeq, nNodes)
        for i in range(data.shape[0]):
            data[i,:,:] = S[i:i+15, :]
        
        torch.save(torch.Tensor(data), self.processed_file_names[0])
        print("Process done.")
        
    def clean(self):
        os.remove('data_sz_GRU.pt')
        
    def __len__(self):
        return len(self.index)
    
    def __getitem__(self, idx: int):
        return self.data[idx, :, :]
    
    @property
    def feature_dim(self):
        return None
    
    @property
    def target(self):
        return "speed"
    
    @property
    def index(self):
        return self._index
    
    @index.setter
    def index(self, indices):
        self._index = indices
    
    @property
    def X(self):
        pass
    
    @property
    def Y(self):
        pass
    
    def loader(self, batch_size):
        def collate_fn(batch):
            batchTS = torch.stack(batch)  # (batch_size, nSeq, nNodes)
            return batchTS[:, :12, :].permute(0,2,1).contiguous(), batchTS[:, 12:, :].permute(0,2,1).contiguous()
        #     return batchTS[:, :12, :], batchTS[:, 12:, :].permute(0, 2, 1).contiguous()
        #     return batchTS[:, :12, :].permute(0,2,1).contiguous(), batchTS[:, 12:, :].permute(0,2,1).contiguous()# .view(batchTS.shape[0], -1)
        return torch.utils.data.DataLoader(self, batch_size=batch_size, collate_fn=collate_fn)
    
    def scale(self, inverse=False):
        assert not self.is_subset, 'This dataset only suport global scale!'
        self.data = self.scalerX(self.data, along=self.feature_dim, inverse=inverse)
    
    def stat(self):
        print(f"mean = {self.data.mean()}")

In [7]:
dataset = SZD4GRU()

In [8]:
# dts_train, dts_valid, dts_test = dataset.random_split([0.7, 0.2, 0.1])
dts_train, dts_valid = dataset.random_split([0.8, 0.2])

In [9]:
dts_valid.stat()

mean = 12.19403076171875


In [10]:
dlr_train = dts_train.loader(batch_size=64)

In [11]:
a, b = next(iter(dlr_train))

In [12]:
a.shape, b.shape

(torch.Size([64, 156, 12]), torch.Size([64, 156, 3]))

# 2. 然后构建net

+ 这里又去看了TGCN里的code半天, 发现他的本质其实是input_dim为1的, hidden到100, output再到3, 如此而已
+ 证据就是做hidden_state的时候, i和h要加的嘛, 他就1+的, 另外, hidden_state的值, 就是nNodes * (1+hidden_dim)
+ 所以nNodes还是充当了batch的角色, 额, 但是他不考虑相互关系就可以做到这么好的效果?试试吧先...

In [13]:
nNodes = dataset.nNodes
nNodes

156

In [14]:
class GRU(nn.Module):
    def __init__(self, nNodes, input_dim, GRU_dim, output_dim):
        super().__init__()
        self.GRULayer = nn.GRU(input_dim, GRU_dim, batch_first=True)
        self.Regressor = nn.Linear(GRU_dim, output_dim)
        
    def forward(self, inputs):
        batch_size, num_nodes, seq_len = inputs.shape
#         h0 = torch.zeros(batch_size, num_nodes * self.GRU_dim).type_as(
#             inputs
#         )
#         print(inputs.shape)
        out, h_gru = self.GRULayer(inputs.view(batch_size * num_nodes, seq_len, 1))  # h_gru : (1, batchsize, hidden_dim)
#         print(h_gru.shape)
        x = self.Regressor(h_gru.squeeze(0))
        return x.view(batch_size, num_nodes, -1)  # x(batch_size * num_nodes, output_dim)

## 测试一下net的forward, 检查shape

In [15]:
net = GRU(nNodes, 1, 100, 3)

In [16]:
ya = net(a)
assert(ya.shape == b.shape), 'Shape differs'
ya.shape, b.shape

(torch.Size([64, 156, 3]), torch.Size([64, 156, 3]))

## 没有问题, 下面开始搭框架

In [17]:
next(net.parameters()).device

device(type='cpu')

In [18]:
net.to(device)

GRU(
  (GRULayer): GRU(1, 100, batch_first=True)
  (Regressor): Linear(in_features=100, out_features=3, bias=True)
)

In [19]:
next(net.parameters()).device

device(type='cuda', index=0)

In [20]:
losser = torch.nn.MSELoss()
# optim = torch.optim.Adam(net.parameters())
optim = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1.5e-3)

esp = air.utils.EarlyStopping(30)

trainer = air.airTrainer(net, criterion=losser, optim=optim, esp=esp, max_iter=30)

In [21]:
apf = air.airPerf(task='regression')

In [22]:
runner = air.airRunner(trainer)

In [23]:
runner.split_run(dataset, apf, learn_ratio=1, train_core_params={
    'batch_size': 64,
    'verbose_nEpoch': 10
})

>>> repeat time 0 @ 2023/10/29 16:45:43
Hint: feature_dim is None in evaluation
[ 0/30] train_loss: 291.5840, valid_loss: 406.6052, min_loss : 406.6052, @2023/10/29 16:45:45
[10/30] train_loss: 64.4351, valid_loss: 124.2020, min_loss : 124.2020, @2023/10/29 16:45:54
[20/30] train_loss: 45.2829, valid_loss: 88.2626, min_loss : 88.2626, @2023/10/29 16:46:03


In [24]:
dts_valid.stat()

mean = 12.19403076171875
